In [2]:
# import stuff
import sys,os,inspect
import numpy as np

%load_ext autoreload
%autoreload 2

sys.path.append( os.environ["PYTHONSCRIPT_DIR"] )

import matplotlib as mpl
import PlotSettings as pS
pS.loadPlotSettings(mpl)

lineSettings = pS.generalSettings.defaultLineSettings 
defaultMarkerSize = pS.generalSettings.defaultMarkerSize

from Tools.Parsers.Utilities import parseNDArray

sys.path.append( os.environ["JOBGENERATOR_DIR"])
from JobGenerator.jobGenerators.commonFunctions import CommonFunctions as cf 

import glob2,re
import scipy
import scipy.interpolate
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from skimage.io import imread
from skimage import img_as_float


#GUi shit
%matplotlib tk
#mpl.get_configdir()

def getScriptPath():
      return os.path.dirname(inspect.getfile(inspect.currentframe()))


Setting matplotlib settings from file: /home/zfmgpu/Desktop/Repository/SimulationFramework/SourceCode/Projects/SimulationFramework/Simulations/PythonScripts/PlotSettings/matplotlibrc


In [3]:
# Defintions

def getAllFiles(globEx,regex, key=lambda m: int(m.group(1))):
    files = glob2.glob(globEx)
    l={}
    for f in files:
        try:
            l[ key(re.match(regex,f)) ] = f
        except:
            pass
    if not l:
        raise ValueError("No files found for %s " % globEx)
        
    return l

expRegex = r".*?experiment(\d*)"
imageRegex = r".*?-(\d*).*"


dataFolder = "/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data"
expRepoDir = os.path.join(getScriptPath(),"../")

outputFolder = os.path.join(expRepoDir,"data/experiment%i")

expSettingsFile = os.path.join(expRepoDir,"data/ExperimentSettings.json")
expSettings = cf.jsonLoad(expSettingsFile)
expDeltaT = 1.0/expSettings["framerate"]

expIntensitySettingsFile = os.path.join(expRepoDir,"data/IntensitySettings.json")
expIntSettings = cf.jsonLoad(expIntensitySettingsFile)

expFolders = getAllFiles(os.path.join(dataFolder,"experiment*"),expRegex )
print(expFolders)

# Plots
# figure stuff
figureSize= (pS.cm2inch(8),pS.cm2inch(6))
legendProps = dict(prop=dict(size=10),borderpad=0.2,labelspacing=0.1)



{1: '/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment1', 2: '/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment2', 3: '/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment3', 4: '/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment4', 5: '/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment5', 6: '/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment6', 7: '/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment7', 8: '/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment8', 9: '/media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment9'}


In [4]:
# Compute Intensity values for all simulations
def computeIntensity(expFolder):

    images= getAllFiles(os.path.join(expFolder,"*"), imageRegex)
    images= sorted(images.items(),key=lambda x:x[0])
    # compute intensity over all orig images
    
    frameIdx = [i[0] for i in images]
    intensity = np.ndarray((len(frameIdx)))
    
    for i,(idx,f) in enumerate(images):
        # make gray image by averaging (as_grey=True does something else -> rgb2grey in skimage)
        im = np.average(img_as_float(imread(f)),2)
        intensity[i]=np.mean(im)
    
    return frameIdx,intensity


def smoothIntensity(frameIdx,intensity,outFolder,expNr):
    
    import scipy.fftpack
    eS = expIntSettings["experiments"][str(expNr)]
    tVec = eS["intensitySmoothSplineTVec"]

    def extendBoundary(x,y, n=100):
        x = np.hstack([np.array(range(x[0]-n,x[0]),dtype=int), 
                      x
                       ])
    #     , 
    #                   np.array(range(x[-1]+1,x[-1]+1+n),dtype=int)
    #                   ])

        y = np.hstack([y[n:0:-1], y ]) #, y[-1:-1-n:-1] ])

        return x,y
    
    
    frameIdxE , intensityE = extendBoundary(frameIdx,intensity)

    w = scipy.fftpack.rfft(intensityE)
    freq = scipy.fftpack.rfftfreq(len(intensityE), expDeltaT)
    spectrum = w**2
    w2 = w.copy()
    zeroing = np.where((freq > 90) & (freq<120))
    w2[zeroing]  = 0

    # plot spectrum
    fig=plt.figure(figsize=figureSize)
    ax=plt.subplot()
    ax.plot(freq,spectrum,lw=lineSettings["extra-thin"],color='k')
    ax.set_yscale("log")
    ax.set_xlabel(r"frequency [\si{\hertz}]")
    ax.set_ylabel(r"$|\mathcal{F}(\Mean{\vI(t)})|^2$")

    x=100
    m=np.abs(freq-x)>1e-3
    y=np.argmax(np.ma.masked_array(spectrum,mask=m))
    ax.text(x-50,spectrum[y],"luminosity freq. of spotlight \n"+r"$=\SINumN{100}{\hertz}$",fontsize=10)
    ax.margins(0.05)
    import matplotlib.ticker as plticker
    loc = plticker.LogLocator(numticks=5) # this locator puts ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    pS.defaultFormatAxes(ax)
    fig.tight_layout(pad=0.2)
    fig.savefig(os.path.join(outFolder,"intensitySpectrum.pdf"))
    rectIntenstiy = scipy.fftpack.irfft(w2)

    # secondPart = 340
    # overlap = 0
    # start = secondPart - overlap 
    # intensitySmoothed = scipy.signal.savgol_filter(reconstructInt[start:],391,5)
    # # plt.plot(frameIdxE[start:],intensitySmoothed)
    # end = secondPart + overlap 
    # intensitySmoothed2 = scipy.signal.savgol_filter(reconstructInt[0:end],281,3)
    # # plt.plot(frameIdxE[0:end],intensitySmoothed2)

    t= np.hstack( [np.linspace(*t) for t in tVec])
    #print("Frame Indices:", frameIdx)
    print("Interpolate intensity with knots: ", t)
    spline = scipy.interpolate.LSQUnivariateSpline(frameIdxE, intensityE,t,k=2)
    sIntensity = spline(frameIdx)
    
    
    from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, inset_axes
    from mpl_toolkits.axes_grid1.inset_locator import mark_inset
    fig, ax = plt.subplots(figsize=figureSize)
    ax.set_xlabel(r"time $t$ [\si{\second}]")
    ax.set_ylabel(r"image mean value $\Mean{\vI(t)}$")
    time = frameIdx.astype(float)*expDeltaT
    ax.plot(time, intensity, lw=lineSettings["extra-thin"], color='k',zorder=1,label="original")
    ax.plot(time, sIntensity,lw=lineSettings["thin"],color="0.7",zorder=2,label="smoothed")
    ax.scatter(t*expDeltaT,spline(t),c="k",marker="o",s=defaultMarkerSize,zorder=3)
    ax.set_xlim(min(time),max(time))
    ax.margins(None,0.05)
    ax.legend(loc=4,**legendProps)
#     loc = plticker.LinearLocator(numticks=5) # this locator puts ticks at regular intervals
#     ax.yaxis.set_major_locator(loc)
    
    axins = inset_axes(ax, width=0.7,height=0.7, loc=2) # zoom = 6
    axins.plot(time,intensity,lw=lineSettings["extra-thin"], color='k',zorder=1)
    axins.plot(time,sIntensity,lw=lineSettings["thin"],color="0.5",zorder=2)
    axins.scatter(t*expDeltaT,spline(t),c="k",marker="o",s=defaultMarkerSize,zorder=3)

    axins.set_xlim(*eS["intensityZoomWindow"][0])
    axins.set_ylim(*eS["intensityZoomWindow"][1])
    axins.set_xticklabels([])
    axins.set_yticklabels([])
    mark_inset(ax, axins, loc1=2, loc2=4, fc="none", lw=lineSettings["thin"], ec="0", zorder=4)
    pS.defaultFormatAxes(ax)
    fig.tight_layout(pad=0.2)
    fig.savefig(os.path.join(outFolder,"intensity.pdf"))
    
    return sIntensity

In [21]:
plt.close("all")

expList = [4,5]

for nr,expFolder in expFolders.items():
    
    if nr in expList:
        
        print("Generate values for data in %s" % expFolder)
        
        outFolder = outputFolder % nr

        if not os.path.exists(outFolder):
            os.makedirs(outFolder)
        
        file = os.path.join(outFolder,"intensityValues.npz")
        if not os.path.exists(file):
            print("Compute Intensity")
            frameIndices, intensity = computeIntensity(expFolder)
            np.savez(file,
                     intensity=intensity, 
                     frameIndices=frameIndices)
        else:
            l=np.load(file)
            frameIndices =  l["frameIndices"]
            intensity =  l["intensity"]
        
        
        
        file = os.path.join(outFolder,"smoothIntensityValues.npz")
        if not os.path.exists(file):
            print("Smooth Intensity")
            sIntensity = smoothIntensity(frameIndices,intensity,outFolder,nr)
            intensityDiff = intensity - sIntensity
            
            np.savez(file,smoothIntensity=sIntensity, 
                     frameIndices=frameIndices, 
                     intensityDiff = intensityDiff)
            
            f=open(os.path.join(outFolder,"intensityShift.json"),"w")
            cf.jsonDump([[fIdx,s] for fIdx,s in zip(frameIndices.tolist(),intensityDiff.tolist())], f)
            
        else:
            l=np.load(file)
            frameIndices =  l["frameIndices"]
            sIntensity   =  l["smoothIntensity"]
            intensityDiff   =  l["intensityDiff"]
        
    
   


Generate values for data in /media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment4
Generate values for data in /media/zfmgpu/MyDataRaid/GabrielNuetzi/ChuteExperimentsAdd/data/experiment5


In [27]:
import h5py
# convert all pictures experimt 5 and animate

# expFolder=expFolders[5]
# print(expFolder)

# images= getAllFiles(os.path.join(expFolder,"*"), imageRegex)
# images= sorted(images.items(),key=lambda x:x[0])
# # compute intensity over all orig images

# frameIdx = [i[0] for i in images]

images= getAllFiles("/media/zfmgpu/MyDataRaid/GabrielNuetzi/CIVExperiments/CIVExperiments-3Passes/CIV-Exp5-4Frames/imageFiles/CIVFine*", ".*?-f-(\d*).*")
images= sorted(images.items(),key=lambda x:x[0])

imageList = images[200:500]
loadedImages = []
for i,(idx,f) in enumerate(imageList):
    # make gray image by averaging (as_grey=True does something else -> rgb2grey in skimage)
#     loadedImages.append( np.average(img_as_float(imread(f)),2) )
    f=h5py.File(f,"r")
    loadedImages.append(f["data"][...])

fig = plt.figure()
im=plt.imshow(loadedImages[0],animated=True,cmap=cm.Greys_r)

# initialization function: plot the background of each frame
def init():
    im.set_data(loadedImages[0])
    return [im]

# animation function.  This is called sequentially
def animate(i):
    im.set_array(loadedImages[i])    # exponential decay of the values
    plt.draw()
    return [im]

from matplotlib import animation
anim = animation.FuncAnimation(fig, animate,
                               frames=len(loadedImages), interval=0.1, blit=True)